<a href="https://colab.research.google.com/github/louis-zoubt/Quantitative_Investment_Strategy/blob/main/BAFI_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import time
import statsmodels.api as sm
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
path_data = '/content/drive/My Drive/BAFI 508 Final project/Data/'


In [ ]:
os.listdir(path_data)


['Compustat_Annual_2018.csv',
 'F-F_Research_Data_Factors_2018.csv',
 'CRSP_Monthly_2018.csv',
 'CRSP_Monthly_2022.csv',
 'F-F_Research_Data_Factors.CSV',
 'Compustat_Annual_2023.csv',
 'Compustat_Annual_2023_1.csv',
 'Compustat_Annual_2023_2.csv']

In [ ]:

#%%############################################################################
# Step 1: Preparing the CRSP file
###############################################################################
print("Prepare CRSP file")
t = time.time() # record the current time, so we can measure how long the code takes to run
crsp = pd.read_csv(path_data+'CRSP_Monthly_2022.csv')
# load data
# Have a look at the data
print(crsp.head())
print(crsp.dtypes)

Prepare CRSP file
   PERMNO        date  SHRCD  EXCHCD TICKER  HSICMG  FACPR  FACSHR     PRC  \
0   10000  1985-12-31    NaN     NaN    NaN     NaN    NaN     NaN     NaN   
1   10000  1986-01-31   10.0     3.0  OMFGA     NaN    NaN     NaN -4.3750   
2   10000  1986-02-28   10.0     3.0  OMFGA     NaN    NaN     NaN -3.2500   
3   10000  1986-03-31   10.0     3.0  OMFGA     NaN    NaN     NaN -4.4375   
4   10000  1986-04-30   10.0     3.0  OMFGA     NaN    NaN     NaN -4.0000   

         RET  SHROUT  
0        NaN     NaN  
1          C  3680.0  
2  -0.257143  3680.0  
3   0.365385  3680.0  
4  -0.098592  3793.0  
PERMNO      int64
date       object
SHRCD     float64
EXCHCD    float64
TICKER     object
HSICMG    float64
FACPR     float64
FACSHR    float64
PRC       float64
RET        object
SHROUT    float64
dtype: object


In [ ]:



### formatting ###
# make all variable names lowercase
crsp.columns = map(str.lower,crsp.columns)

# You should see that one of the important variables 'RET' (return) is not a number but 'object'.
# It is preferable to have this variable as a number, which Python denotes as float64 (float64 is just a special way of saying that a variable is a number)
# If you are interested search for 'floating point number'on internet. But it is computer-science issue!

# Changes the returns to number format. Non-numeric data will be NAN
crsp['ret'] = pd.to_numeric(crsp['ret'],errors='coerce')

# Change the dateformat
crsp['date'] = pd.to_datetime(crsp['date'], format='%Y/%m/%d')

# Create separate 'year' and 'month' variables (we will use them later to merge CRSP with Compustat)
crsp['year'] = crsp['date'].apply(lambda date: date.year)
crsp['month'] = crsp['date'].apply(lambda date: date.month)

# Calculate market cap
crsp['mktcap'] = crsp['shrout'] * crsp['prc'].abs()



In [ ]:

### Some basic data cleaning ###
# keep only common shares
crsp = crsp[crsp['shrcd'].isin([10,11])]

# keep only stocks from NYSE, AMEX and NASDAQ
crsp = crsp[crsp['exchcd'].isin([1,2,3])]

# make sure that there are no duplicates
# usually, we would investigate why there are duplicates and then decide which observation we want to keep
#    For here, it is enough to simply drop the duplicates.
crsp = crsp.drop_duplicates(subset=['date','permno'])


print('Completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block


Completed in 28.4s


In [ ]:
#%%############################################################################
# Step 2: Preparing the Compustat (CCM) file
###############################################################################
print("Prepare Compustat file")
t = time.time() # reset our timer

ccm = pd.read_csv(path_data+'Compustat_Annual_2023_2.csv')

# Have a look at the data
print(ccm.head())
print(ccm.dtypes)
print(ccm.shape)



Prepare Compustat file
   GVKEY LINKTYPE  LPERMNO  LPERMCO    datadate   fyear indfmt consol popsrc  \
0   1000       LU    25881    23369  1970-12-31  1970.0   INDL      C      D   
1   1000       LU    25881    23369  1971-12-31  1971.0   INDL      C      D   
2   1000       LU    25881    23369  1972-12-31  1972.0   INDL      C      D   
3   1000       LU    25881    23369  1973-12-31  1973.0   INDL      C      D   
4   1000       LU    25881    23369  1974-12-31  1974.0   INDL      C      D   

  datafmt                   conm curcd      at   capx   csho  epspx costat  \
0     STD  A & E PLASTIK PAK INC   USD  33.450  2.767  2.446   0.56      I   
1     STD  A & E PLASTIK PAK INC   USD  29.330  1.771  2.995   0.04      I   
2     STD  A & E PLASTIK PAK INC   USD  19.907  1.254  2.902   0.50      I   
3     STD  A & E PLASTIK PAK INC   USD  21.771  1.633  2.840   0.64      I   
4     STD  A & E PLASTIK PAK INC   USD  25.638  1.313  2.150   0.60      I   

   prcc_c  mkvalt   sic  
0

In [ ]:


### formatting ###
# make all variable names lowercase
ccm.columns = map(str.lower,ccm.columns)

# Change the dateformat
ccm['datadate'] = pd.to_datetime(ccm['datadate'], format='%Y-%m-%d')

# Create separate 'year' and 'month' variables
ccm['year'] = ccm['datadate'].apply(lambda x: x.year)
ccm['month'] = ccm['datadate'].apply(lambda x: x.month)

### Some basic data cleaning ###
# make sure that there are no duplicates (same as above)
ccm = ccm.drop_duplicates(subset=['datadate','gvkey'])
ccm = ccm.drop_duplicates(subset=['year','gvkey'])
ccm = ccm.drop_duplicates(subset=['year','lpermno'])




In [ ]:
### Calculate the variables we will use for sorting ###
# Create earnings to price ratio variable

ccm = ccm.sort_values(['gvkey','datadate']) # sort data by gvkey and date
#
ccm['e/p'] = ccm['epspx'] / ccm['prcc_c']
# added market capitalization variable = closing price * number of shares
ccm['mktcap'] = ccm['csho'] * ccm['prcc_c']
# It is useful to know how many observations are missing
print('Fraction of observations missing:')
print(1 - ccm.count() / len(ccm))


print('Completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block



Fraction of observations missing:
gvkey       0.000000
linktype    0.000000
lpermno     0.000000
lpermco     0.000000
datadate    0.000000
fyear       0.000006
indfmt      0.000000
consol      0.000000
popsrc      0.000000
datafmt     0.000000
conm        0.000000
curcd       0.000006
at          0.066660
capx        0.133841
csho        0.005717
epspx       0.072717
costat      0.000000
prcc_c      0.013062
mkvalt      0.613753
sic         0.000000
year        0.000000
month       0.000000
e/p         0.084120
mktcap      0.017839
dtype: float64
Completed in 3.0s


In [ ]:
#%%############################################################################
# Step 3: Sort stocks into portfolios and calculate returns
###############################################################################
print("Create portfolios by sorting by earning to price and then size of the company")
t = time.time() # reset our timer

# loop over all years in the data
# Note: the first loop loops over the years in range(1951,2023).
#    You can wrap any list by the tqdm command to display a progress bar while looping over the list
portfolios = [] # create an empty list to collect the portfolio returns
for year in tqdm(range(1963,2023),desc="years"):
    # take the companies that were alive at t-1
    permno_list=list(crsp[crsp['year']==year-1]['permno'].unique())

    # get the sorting variable for these companies at t-1
    sorting_data = ccm.loc[(ccm['year']==(year-1)) & \
                           (ccm['lpermno'].isin(permno_list)), \
                           ['gvkey','lpermno','e/p', 'mktcap']]

    # sort into 5 baskets by cashflow over assets
    nportfolios = 5 # number of portfolios
    sorting_data['rank'] = pd.qcut(sorting_data['e/p'],nportfolios, labels=False)
    # add market captilization layer, each e/p bin divide by market captilization
    for n in range(nportfolios):
      sorting_data.loc[sorting_data['rank'] == n, 'rank1'] = pd.qcut(sorting_data.loc[sorting_data['rank'] == n, 'mktcap'], nportfolios, labels = False)

    # select the return data with some time lag to make sure that the accounting information is public (data from July at year t to June in year t+1)
    crsp_window = crsp[((crsp['year']==year) & (crsp['month']>=6)) | \
                       ((crsp['year']==year+1) & (crsp['month']<=6))]

    # create the portfolio returns for the current window and collect them in portfolios_window
    portfolios_window = []
    for p in range(nportfolios):
       for n in range(nportfolios):
          # get list of permnos that are in this portfolio
          basket = sorting_data.loc[(sorting_data['rank'] == p) & (sorting_data['rank1'] == n),'lpermno'].tolist()

          # get returns of these permnos
          crsp_p_firms = crsp_window[crsp_window['permno'].isin(basket)]

          # pivot returns
          returns = crsp_p_firms.pivot(index='date', columns='permno', values='ret')
          returns = returns.iloc[1:,:] # drop the first row

          # create equally weighted portfolio (monthly rebalancing)
          return_port = returns.mean(axis=1)
          return_port.name = str(p) + '.' + str(n) # naming

          # collect portfolio returns in dec_port
          portfolios_window += [return_port]

    # merge the portfolios
    portfolios_window = pd.concat(portfolios_window,axis=1)

    # collect results in portfolios
    portfolios += [portfolios_window]

# merge the returns from all windows
portfolios = pd.concat(portfolios,axis=0)


print('Step 3 completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block



Create portfolios by sorting by earning to price and then size of the company


years: 100%|██████████| 60/60 [00:09<00:00,  6.62it/s]

Step 3 completed in 9.1s


In [ ]:
# Create long-short portfolios for each segment
portfolios['0.5'] = portfolios['0.4'] - portfolios['0.0']
portfolios['1.5'] = portfolios['1.4'] - portfolios['1.0']
portfolios['2.5'] = portfolios['2.4'] - portfolios['2.0']
portfolios['3.5'] = portfolios['3.4'] - portfolios['3.0']
portfolios['4.5'] = portfolios['4.4'] - portfolios['4.0']


In [ ]:
# monthly mean return
print(portfolios.mean(axis=0))

0.0    0.023065
0.1    0.016125
0.2    0.011588
0.3    0.009642
0.4    0.008044
1.0    0.015510
1.1    0.009770
1.2    0.008069
1.3    0.006713
1.4    0.007430
2.0    0.014302
2.1    0.011376
2.2    0.010332
2.3    0.010244
2.4    0.009401
3.0    0.014978
3.1    0.012398
3.2    0.012863
3.3    0.012298
3.4    0.011737
4.0    0.017992
4.1    0.014276
4.2    0.014270
4.3    0.014315
4.4    0.012486
0.5   -0.015021
1.5   -0.008079
2.5   -0.004901
3.5   -0.003241
4.5   -0.005506
dtype: float64


In [ ]:
#%%############################################################################
# Step 3.1: Sort stocks into portfolios and calculate returns
###############################################################################
print("Create portfolios by first sorting by size then sorting by earning to price")
t = time.time() # reset our timer

# loop over all years in the data
portfolios_1 = [] # create an empty list to collect the portfolio returns
for year in tqdm(range(1963,2023),desc="years"):
    # take the companies that were alive at t-1
    permno_list=list(crsp[crsp['year']==year-1]['permno'].unique())

    # get the sorting variable for these companies at t-1
    sorting_data = ccm.loc[(ccm['year']==(year-1)) & \
                           (ccm['lpermno'].isin(permno_list)), \
                           ['gvkey','lpermno','e/p', 'mktcap']]

    # sort into 5 baskets by cashflow over assets
    nportfolios = 5 # number of portfolios
    sorting_data['rank'] = pd.qcut(sorting_data['mktcap'],nportfolios, labels=False)

    for n in range(nportfolios):
      sorting_data.loc[sorting_data['rank'] == n, 'rank1'] = pd.qcut(sorting_data.loc[sorting_data['rank'] == n, 'e/p'], nportfolios, labels = False)

    # select the return data with some time lag to make sure that the accounting information is public (data from July at year t to June in year t+1)
    crsp_window = crsp[((crsp['year']==year) & (crsp['month']>=6)) | \
                       ((crsp['year']==year+1) & (crsp['month']<=6))]

    # create the portfolio returns for the current window and collect them in portfolios_window
    portfolios_window_1 = []
    for p in range(nportfolios):
       for n in range(nportfolios):
          # get list of permnos that are in this portfolio
          basket = sorting_data.loc[(sorting_data['rank'] == p) & (sorting_data['rank1'] == n),'lpermno'].tolist()

          # get returns of these permnos
          crsp_p_firms = crsp_window[crsp_window['permno'].isin(basket)]

          # pivot returns
          returns = crsp_p_firms.pivot(index='date', columns='permno', values='ret')
          returns = returns.iloc[1:,:] # drop the first row

          # create equally weighted portfolio (monthly rebalancing)
          return_port = returns.mean(axis=1)
          return_port.name = str(p) + '.' + str(n)

          # collect portfolio returns in dec_port
          portfolios_window_1 += [return_port]

    # merge the portfolios
    portfolios_window_1 = pd.concat(portfolios_window_1,axis=1)

    # collect results in portfolios
    portfolios_1 += [portfolios_window_1]

# merge the returns from all windows
portfolios_1 = pd.concat(portfolios_1,axis=0)


print('Step 3 completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block



Create portfolios by first sorting by size then sorting by earning to price


years: 100%|██████████| 60/60 [00:10<00:00,  5.49it/s]

Step 3 completed in 10.9s


In [ ]:
# Create long-short portfolios for each segment
portfolios_1['0.5'] = portfolios_1['0.4'] - portfolios_1['0.0']
portfolios_1['1.5'] = portfolios_1['1.4'] - portfolios_1['1.0']
portfolios_1['2.5'] = portfolios_1['2.4'] - portfolios_1['2.0']
portfolios_1['3.5'] = portfolios_1['3.4'] - portfolios_1['3.0']
portfolios_1['4.5'] = portfolios_1['4.4'] - portfolios_1['4.0']


In [ ]:
print(portfolios_1.mean(axis=0))

0.0    0.018773
0.1    0.018620
0.2    0.018065
0.3    0.017375
0.4    0.017989
1.0    0.012187
1.1    0.009361
1.2    0.011034
1.3    0.013060
1.4    0.013905
2.0    0.009459
2.1    0.008246
2.2    0.011206
2.3    0.012277
2.4    0.013803
3.0    0.008122
3.1    0.009110
3.2    0.010986
3.3    0.011829
3.4    0.013254
4.0    0.007883
4.1    0.009173
4.2    0.009690
4.3    0.011078
4.4    0.012048
0.5   -0.000784
1.5    0.001717
2.5    0.004343
3.5    0.005132
4.5    0.004165
dtype: float64


In [ ]:
#%%############################################################################
# Step 4: Performance Evaluation
# Step 4a: Merge Portfolio returns with Fama French data
###############################################################################

### load and prepare fama french data ###
# load Fama French monthly factors
ff = pd.read_csv(path_data+'F-F_Research_Data_Factors.CSV')

# rename columns
ff.rename({'Mkt-RF':'ExMkt',
           'DATE':'date'},axis=1,inplace=True)

# date variables
ff['year'] = ff['date'] // 100
ff['month'] = ff['date'] % 100
ff.set_index('date',inplace=True)


### formatting ###
# FF data is in percent. Convert to simple returns
ff[['ExMkt', 'SMB', 'HML', 'RF']] /= 100


In [ ]:
print(range(nportfolios))

range(0, 5)


In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:

### merge portfolio returns with Fama French data ###
# date variables
portfolios_ff = portfolios.copy() # create a copy of the portfolios dataframe so we can use it again later
portfolios_ff['year'] = portfolios_ff.index.year
portfolios_ff['month'] = portfolios_ff.index.month

# merge
portfolios_ff = pd.merge(portfolios_ff,ff,on=['year','month'])
# calculate excess returns
for p in range(nportfolios):
  for n in range(nportfolios):
    portfolios_ff['excess'+str(p)+'.'+str(n)] = portfolios_ff[str(p)+'.'+str(n)] - portfolios_ff['RF']

In [ ]:
### merge portfolio returns with Fama French data ###
# date variables
portfolios_ff_1 = portfolios_1.copy() # create a copy of the portfolios dataframe so we can use it again later
portfolios_ff_1['year'] = portfolios_ff_1.index.year
portfolios_ff_1['month'] = portfolios_ff_1.index.month

# merge
portfolios_ff_1 = pd.merge(portfolios_ff_1,ff,on=['year','month'])
# calculate excess returns
for p in range(nportfolios):
  for n in range(nportfolios):
    portfolios_ff_1['excess'+str(p)+'.'+str(n)] = portfolios_ff_1[str(p)+'.'+str(n)] - portfolios_ff['RF']

In [ ]:
print(portfolios_ff.mean(axis=0))


0.0             0.023065
0.1             0.016125
0.2             0.011588
0.3             0.009642
0.4             0.008044
1.0             0.015510
1.1             0.009770
1.2             0.008069
1.3             0.006713
1.4             0.007430
2.0             0.014302
2.1             0.011376
2.2             0.010332
2.3             0.010244
2.4             0.009401
3.0             0.014978
3.1             0.012398
3.2             0.012863
3.3             0.012298
3.4             0.011737
4.0             0.017992
4.1             0.014276
4.2             0.014270
4.3             0.014315
4.4             0.012486
0.5            -0.015021
1.5            -0.008079
2.5            -0.004901
3.5            -0.003241
4.5            -0.005506
year         1992.747899
month           6.525210
ExMkt           0.005495
SMB             0.001853
HML             0.003115
RF              0.003621
excess0.0       0.019445
excess0.1       0.012504
excess0.2       0.007967
excess0.3       0.006021


In [ ]:
print(portfolios_ff_1.mean(axis=0))

0.0             0.018773
0.1             0.018620
0.2             0.018065
0.3             0.017375
0.4             0.017989
1.0             0.012187
1.1             0.009361
1.2             0.011034
1.3             0.013060
1.4             0.013905
2.0             0.009459
2.1             0.008246
2.2             0.011206
2.3             0.012277
2.4             0.013803
3.0             0.008122
3.1             0.009110
3.2             0.010986
3.3             0.011829
3.4             0.013254
4.0             0.007883
4.1             0.009173
4.2             0.009690
4.3             0.011078
4.4             0.012048
0.5            -0.000784
1.5             0.001717
2.5             0.004343
3.5             0.005132
4.5             0.004165
year         1992.747899
month           6.525210
ExMkt           0.005495
SMB             0.001853
HML             0.003115
RF              0.003621
excess0.0       0.015152
excess0.1       0.014999
excess0.2       0.014444
excess0.3       0.013754


In [ ]:
print("Average returns (annualized percent) by sorting by E/P and then Size\n",((1+portfolios_ff.iloc[:,-25:].mean(axis=0))**12-1)*100)
print("Average returns of long-short portfolios within each E/P bin(annualized percent)\n",((1+portfolios_ff.iloc[:,25:30].mean(axis=0))**12-1)*100)


Average returns (annualized percent) by sorting by E/P and then Size
 excess0.0    25.997879
excess0.1    16.080895
excess0.2     9.990872
excess0.3     7.469282
excess0.4     5.438999
excess1.0    15.237701
excess1.1     7.633919
excess1.2     5.471159
excess1.3     3.774234
excess1.4     4.668449
excess2.0    13.598545
excess2.1     9.714538
excess2.2     8.357303
excess2.3     8.244640
excess2.4     7.161770
excess3.0    14.512784
excess3.1    11.056495
excess3.2    11.672690
excess3.3    10.924447
excess3.4    10.186206
excess4.0    18.676375
excess4.1    13.562693
excess4.2    13.554450
excess4.3    13.616243
excess4.4    11.173227
dtype: float64
Average returns of long-short portfolios within each E/P bin(annualized percent)
 0.5   -16.608395
1.5    -9.275941
2.5    -5.725151
3.5    -3.820397
4.5    -6.410489
dtype: float64


In [ ]:
print("Average returns (annualized percent) by sorting by size and then E/P\n",((1+portfolios_ff_1.iloc[:,-25:].mean(axis=0))**12-1)*100)
print("Average returns of long-short portfolios within each size bin(annualized percent)\n",((1+portfolios_ff_1.iloc[:,25:30].mean(axis=0))**12-1)*100)
# small cap and large E/P

Average returns (annualized percent) by sorting by size and then E/P
 excess0.0    19.777135
excess0.1    19.560553
excess0.2    18.778261
excess0.3    17.812405
excess0.4    18.671926
excess1.0    10.778628
excess1.1     7.110313
excess1.2     9.267418
excess1.3    11.934709
excess1.4    13.063577
excess2.0     7.235953
excess2.1     5.693290
excess2.2     9.492501
excess2.3    10.896813
excess2.4    12.926655
excess3.0     5.537286
excess3.1     6.790327
excess3.2     9.206001
excess3.3    10.306784
excess3.4    12.192549
excess4.0     5.236743
excess4.1     6.870484
excess4.2     7.530789
excess4.3     9.325672
excess4.4    10.594973
dtype: float64
Average returns of long-short portfolios within each size bin(annualized percent)
 0.5   -0.936643
1.5    2.080463
2.5    5.338440
3.5    6.335269
4.5    5.113800
dtype: float64


In [ ]:
excess = pd.concat([portfolios_ff.iloc[:,-25:], portfolios_ff.iloc[:,25:30]], axis = 1)

In [ ]:
excess1 = pd.concat([portfolios_ff_1.iloc[:,-25:], portfolios_ff_1.iloc[:,25:30]], axis = 1)

In [ ]:
returns = excess.aggregate(['mean','std','skew','kurt']).T

In [ ]:
returns["mean_annualized"] = ((1+returns["mean"])**12-1)
returns["std_annualized"] = returns["std"]*np.sqrt(12)
returns["skewness_annualized"] = returns["skew"]*(1/np.sqrt(12))
returns["kurtosis_annualized"] = returns["kurt"]*(1/12)
returns = returns.T


In [ ]:
returns1 = excess1.aggregate(['mean','std','skew','kurt']).T
returns1["mean_annualized"] = ((1+returns1["mean"])**12-1)
returns1["std_annualized"] = returns1["std"]*np.sqrt(12)
returns1["skewness_annualized"] = returns1["skew"]*(1/np.sqrt(12))
returns1["kurtosis_annualized"] = returns1["kurt"]*(1/12)
returns1 = returns1.T

In [ ]:
from scipy.stats import ttest_1samp
t, p = ttest_1samp(excess, 0)
tvalue = pd.DataFrame([t, p]).transpose().rename(columns={0:"t-stat",1:"p-value"},\
                                                 index={0:"excess0.0",1:"excess0.1",2:"excess0.2",3:"excess0.3",4:"excess0.4",\
                                                        5:"excess1.0",6:"excess1.1",7:"excess1.2",8:"excess1.3",9:"excess1.4",\
                                                        10:"excess2.0",11:"excess2.1",12:"excess2.2",13:"excess2.3",14:"excess2.4",\
                                                        15:"excess3.0",16:"excess3.1",17:"excess3.2",18:"excess3.3",19:"excess3.4",\
                                                        20:"excess4.0",21:"excess4.1",22:"excess4.2",23:"excess4.3",24:"excess4.4",\
                                                        25:"0.5",26:"1.5",27:"2.5",28:"3.5",29:"4.5"}).T
result = pd.concat([returns, tvalue], axis = 0)
print(result)

                        excess0.0  excess0.1  excess0.2  excess0.3  excess0.4  \
mean                 1.944454e-02   0.012504   0.007967   0.006021   0.004423   
std                  9.526282e-02   0.094111   0.092507   0.091098   0.087964   
skew                 9.039765e-01   1.075861   0.764640   0.489386   0.540239   
kurt                 3.173587e+00   4.621454   3.856926   3.032032   3.829977   
mean_annualized      2.599788e-01   0.160809   0.099909   0.074693   0.054390   
std_annualized       3.300001e-01   0.326010   0.320455   0.315571   0.304716   
skewness_annualized  2.609555e-01   0.310574   0.220733   0.141273   0.155954   
kurtosis_annualized  2.644656e-01   0.385121   0.321411   0.252669   0.319165   
t-stat               5.454102e+00   3.550228   2.301322   1.766070   1.343662   
p-value              6.792359e-08   0.000410   0.021661   0.077812   0.179485   

                     excess1.0  excess1.1  excess1.2  excess1.3  excess1.4  \
mean                  0.011889

In [ ]:
t1, p1 = ttest_1samp(excess, 0)
tvalue1 = pd.DataFrame([t1, p1]).transpose().rename(columns={0:"t-stat",1:"p-value"},\
                                                 index={0:"excess0.0",1:"excess0.1",2:"excess0.2",3:"excess0.3",4:"excess0.4",\
                                                        5:"excess1.0",6:"excess1.1",7:"excess1.2",8:"excess1.3",9:"excess1.4",\
                                                        10:"excess2.0",11:"excess2.1",12:"excess2.2",13:"excess2.3",14:"excess2.4",\
                                                        15:"excess3.0",16:"excess3.1",17:"excess3.2",18:"excess3.3",19:"excess3.4",\
                                                        20:"excess4.0",21:"excess4.1",22:"excess4.2",23:"excess4.3",24:"excess4.4",\
                                                        25:"0.5",26:"1.5",27:"2.5",28:"3.5",29:"4.5"}).T
result1 = pd.concat([returns1, tvalue1], axis = 0)
print(result1)

                        excess0.0  excess0.1  excess0.2  excess0.3  excess0.4  \
mean                 1.515220e-02   0.014999   0.014444   0.013754   0.014368   
std                  1.056034e-01   0.091603   0.073873   0.061611   0.058170   
skew                 1.128168e+00   0.969829   0.511375   0.306082   0.134350   
kurt                 4.305342e+00   4.453844   2.529401   3.350877   3.758752   
mean_annualized      1.977714e-01   0.195606   0.187783   0.178124   0.186719   
std_annualized       3.658211e-01   0.317323   0.255903   0.213426   0.201506   
skewness_annualized  3.256742e-01   0.279966   0.147621   0.088358   0.038783   
kurtosis_annualized  3.587785e-01   0.371154   0.210783   0.279240   0.313229   
t-stat               5.454102e+00   3.550228   2.301322   1.766070   1.343662   
p-value              6.792359e-08   0.000410   0.021661   0.077812   0.179485   

                     excess1.0  excess1.1  excess1.2  excess1.3  excess1.4  \
mean                  0.008567

In [ ]:
#%%############################################################################
# Step 4b: Regressions
###############################################################################

# show average returns (annualized and in percent)
print("Average returns (annualized percent)\n",((1+portfolios.mean(axis=0))**12-1)*100)


Average returns (annualized percent)
 0.0    31.473845
0.1    21.161127
0.2    14.826673
0.3    12.203542
0.4    10.091356
1.0    20.284156
1.1    12.374815
1.2    10.124815
1.3     8.359333
1.4     9.289686
2.0    18.579279
2.1    14.539221
2.2    13.127346
2.3    13.010144
2.4    11.883633
3.0    19.530184
3.1    15.935147
3.2    16.576105
3.3    15.797791
3.4    15.029864
4.0    23.860447
4.1    18.541989
4.2    18.533415
4.3    18.597687
4.4    16.056571
0.5   -16.608395
1.5    -9.275941
2.5    -5.725151
3.5    -3.820397
4.5    -6.410489
dtype: float64


In [ ]:
portfolios_ff['excess0.5'] = portfolios_ff['0.5']
portfolios_ff['excess1.5'] = portfolios_ff['1.5']
portfolios_ff['excess2.5'] = portfolios_ff['2.5']
portfolios_ff['excess3.5'] = portfolios_ff['3.5']
portfolios_ff['excess4.5'] = portfolios_ff['4.5']

portfolios_ff_1['excess0.5'] = portfolios_ff_1['0.5']
portfolios_ff_1['excess1.5'] = portfolios_ff_1['1.5']
portfolios_ff_1['excess2.5'] = portfolios_ff_1['2.5']
portfolios_ff_1['excess3.5'] = portfolios_ff_1['3.5']
portfolios_ff_1['excess4.5'] = portfolios_ff_1['4.5']

In [ ]:

### Market model regressions on E/P ~ Size###
table_capm = []
for p in range(nportfolios):
  for n in range(nportfolios + 1):
    # regress portfolio excess return on market excess return
    results = sm.OLS(portfolios_ff['excess'+str(p)+'.'+str(n)],
                     sm.add_constant(portfolios_ff['ExMkt'])).fit()

    # collect results
    table_row = pd.DataFrame({'alpha':results.params['const'],
                              'beta_mkt':results.params['ExMkt'],
                              'alpha_t':results.tvalues['const'],
                              'rmse':np.sqrt(results.mse_resid),
                              'R2':results.rsquared},
                             index=[str(p)+'.'+str(n)])

    table_capm += [table_row]

# Combine the results for all portfolios
table_capm = pd.concat(table_capm,axis=0)
table_capm.index.name = 'quintile'

# show results
print("CAPM\n",table_capm)


### Three Factor model regressions ###
table_ff = []
for p in range(nportfolios):
  for n in range(nportfolios+1):
    # regress portfolio excess return on market excess return
    results = sm.OLS(portfolios_ff['excess'+str(p)+'.'+str(n)],
                     sm.add_constant(portfolios_ff[['ExMkt','SMB','HML']])).fit()

    # collect results
    table_row = pd.DataFrame({'alpha':results.params['const'],
                              'beta_mkt':results.params['ExMkt'],
                              'beta_size':results.params['SMB'],
                              'beta_hml':results.params['HML'],
                              'alpha_t':results.tvalues['const'],
                              'rmse':np.sqrt(results.mse_resid),
                              'R2':results.rsquared},
                             index=[str(p)+'.'+str(n)])

    table_ff += [table_row]

# Combine the results for all portfolios
table_ff = pd.concat(table_ff,axis=0)
table_ff.index.name = 'Portfolio'

# show results
print("Fama-French 3\n",table_ff)


CAPM
              alpha  beta_mkt   alpha_t      rmse        R2
quintile                                                  
0.0       0.013203  1.135836  4.352541  0.080458  0.287669
0.1       0.005712  1.236010  1.993963  0.075984  0.349038
0.2       0.000487  1.361367  0.186053  0.069384  0.438236
0.3      -0.001917  1.444621 -0.795851  0.063887  0.508866
0.4      -0.004022  1.536890 -1.959875  0.054426  0.617713
0.5      -0.017225  0.401054 -6.946176  0.065772  0.070064
1.0       0.006301  1.016987  3.197751  0.052262  0.434175
1.1      -0.000597  1.227835 -0.341974  0.046336  0.587260
1.2      -0.003024  1.359971 -1.854840  0.043242  0.667151
1.3      -0.004441  1.370963 -2.875349  0.040967  0.694132
1.4      -0.003442  1.319725 -2.878970  0.031711  0.778249
1.5      -0.009743  0.302738 -4.891132  0.052834  0.062382
2.0       0.005275  0.983897  3.306445  0.042317  0.522768
2.1       0.001202  1.192726  0.932410  0.034195  0.711433
2.2       0.000169  1.190667  0.155189  0.028812  

In [ ]:

### Market model regressions on E/P ~ Size###
table_capm = []
for p in range(nportfolios):
  for n in range(nportfolios + 1):
    # regress portfolio excess return on market excess return
    results = sm.OLS(portfolios_ff_1['excess'+str(p)+'.'+str(n)],
                     sm.add_constant(portfolios_ff_1['ExMkt'])).fit()

    # collect results
    table_row = pd.DataFrame({'alpha':results.params['const'],
                              'beta_mkt':results.params['ExMkt'],
                              'alpha_t':results.tvalues['const'],
                              'rmse':np.sqrt(results.mse_resid),
                              'R2':results.rsquared},
                             index=[str(p)+'.'+str(n)])

    table_capm += [table_row]

# Combine the results for all portfolios
table_capm = pd.concat(table_capm,axis=0)
table_capm.index.name = 'quintile'

# show results
print("CAPM\n",table_capm)


### Three Factor model regressions ###
table_ff = []
for p in range(nportfolios):
  for n in range(nportfolios+1):
    # regress portfolio excess return on market excess return
    results = sm.OLS(portfolios_ff_1['excess'+str(p)+'.'+str(n)],
                     sm.add_constant(portfolios_ff_1[['ExMkt','SMB','HML']])).fit()

    # collect results
    table_row = pd.DataFrame({'alpha':results.params['const'],
                              'beta_mkt':results.params['ExMkt'],
                              'beta_size':results.params['SMB'],
                              'beta_hml':results.params['HML'],
                              'alpha_t':results.tvalues['const'],
                              'rmse':np.sqrt(results.mse_resid),
                              'R2':results.rsquared},
                             index=[str(p)+'.'+str(n)])

    table_ff += [table_row]

# Combine the results for all portfolios
table_ff = pd.concat(table_ff,axis=0)
table_ff.index.name = 'Portfolio'

# show results
print("Fama-French 3\n",table_ff)


CAPM
              alpha  beta_mkt   alpha_t      rmse        R2
quintile                                                  
0.0       0.008196  1.266045  2.442582  0.088993  0.290838
0.1       0.008354  1.209280  3.004370  0.073754  0.352647
0.2       0.008906  1.007827  4.047227  0.058366  0.376628
0.3       0.008718  0.916491  5.046896  0.045817  0.447766
0.4       0.009474  0.890698  5.954552  0.042200  0.474434
0.5       0.001279 -0.375347  0.525463  0.064537  0.064148
1.0       0.000836  1.406927  0.310518  0.071407  0.440294
1.1      -0.001329  1.286490 -0.672020  0.052434  0.549516
1.2       0.001505  1.075221  1.011384  0.039465  0.600664
1.3       0.004338  0.928511  3.390823  0.033930  0.602776
1.4       0.004933  0.973786  3.628749  0.036059  0.596411
1.5       0.004097 -0.433141  1.912128  0.056836  0.105296
2.0      -0.002511  1.519579 -1.130715  0.058902  0.574227
2.1      -0.002817  1.354306 -1.717020  0.043511  0.662522
2.2       0.001173  1.167098  0.964868  0.032239  

In [ ]:
print(((1+table_capm['alpha'].mean(axis=0))**12-1)*100)

3.5201393493860422


In [ ]:
print(((1+table_ff['alpha'].mean(axis=0))**12-1)*100)

1.4697312394496453
